### IS Proof of concept
Finding Product position on Social Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [5]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [6]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [8]:
from IPython.display import clear_output

In [9]:
# Files definition
root_path = 'Datasource'

comment_file = []
comment_file.append(root_path+"/comment_meiji.csv")
comment_file.append(root_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_path+"/comment_nlpToken.csv"
comment_cooc_nlp_file = root_path+"/comment_cooc.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [10]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]

In [11]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    stopwords.remove('ไม่')
    stopwords.remove('ได้')
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [12]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i).lower())
    return returnList

In [13]:
list_punc = []
for c in string.punctuation:
    list_punc.append(c)

### 1.Clean data & NLP Tokenized

In [14]:
df_textList1 = pd.read_csv(comment_file[0],header=None)
df_textList2 = pd.read_csv(comment_file[1],header=None)

In [15]:
df_textList1.insert(0,'Table','Meiji')
df_textList1.columns = ['Table','URLs', 'text', 'Retrived_date']
df_textList2.insert(0,'Table','DDFC')
df_textList2.columns = ['Table','URLs', 'text', 'Retrived_date']

In [16]:
pd.DataFrame(df_textList1.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,1000.0,43.049,53.682105,3.0,11.0,21.0,49.0,351.0


In [17]:
print('meiji comment:',df_textList1.shape[0])
print('meiji URLs ก่อนตัด:',len(df_textList1.URLs.unique()))

meiji comment: 43049
meiji URLs ก่อนตัด: 1000


In [18]:
pd.DataFrame(df_textList2.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,517.0,44.756286,57.056773,1.0,10.0,19.0,53.0,328.0


In [19]:
print('DDCF comment:',df_textList2.shape[0])
print('DDCF URLs ก่อนตัด:',len(df_textList2.URLs.unique()))

DDCF comment: 23139
DDCF URLs ก่อนตัด: 517


In [20]:
df_Corpus = pd.concat([df_textList1,df_textList2],ignore_index=True)

In [21]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

All comment: 66188
All URLs ก่อนตัด: 1517


In [22]:
missing = df_Corpus.isnull().sum()
print(missing)

Table               0
URLs                0
text             4703
Retrived_date       0
dtype: int64


In [23]:
df_Corpus.tail()

,Table,URLs,text,Retrived_date
66183,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [24]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','Table','URLs', 'text', 'Retrived_date']

In [25]:
df_Corpus.tail()

,commentId,Table,URLs,text,Retrived_date
66183,66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,66188,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [26]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [27]:
missing = df_Corpus.isnull().sum()
print(missing)

Table            0
URLs             0
text             0
Retrived_date    0
dtype: int64


In [28]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [29]:
df_Corpus.insert(4,'token_text',None)
#df_Corpus.insert(5,'token_sound',None)
#df_Corpus.insert(6,'token_tag',None)
df_Corpus.tail()

,Table,URLs,text,Retrived_date,token_text
commentId,,,,,
66181,DDFC,https://pantip.com/topic/30651064,นมผง คือเอานมดิบดีๆ มาคัดไขมันนมและอื่นๆออกไปใ...,2020-10-27 13:54:33.920930,None
66182,DDFC,https://pantip.com/topic/30651064,นมสด รสชาติมันดีนะ\r\nเพราะมีปริมาณไขมันสูง ดั...,2020-10-27 13:54:33.920930,None
66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930,None
66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930,None
66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930,None


เก็บเฉพาะหัวกระทู้

In [30]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','text']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_text',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

,URLs,text,token_text,mention_product,mention_brand
commentId,,,,,
65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,None,None,None
66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,None,None,None
66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...",None,None,None
66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,None,None,None
66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,None,None,None


## 2.ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [31]:
garbage_char = ['',' ','🥺','\r','\n','\r\n','\xa0','www','application','ทางกฎหมาย','html'
                ,'http','https','url','com','pantip','topic','_','–','พันทิป','^^'
                ,'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ','แฮ่','เย่ๆๆ','ว๊าว','ฮิ้วววววววววว','Number','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','(',')','ฯ','ๆ','กก','อันนี้','%s','😆', '🤣','😢'
                ,'😏','😂','😿','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','://','มกราคม', 'กุมภาพันธ์'
                ,'มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม', 'สิงหาคม','กันยายน', 'ตุลาคม'
                ,'พฤศจิกายน', 'ธันวาคม']
garbage_char  = garbage_char + list_punc

In [32]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [33]:
custom_list = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','betagen','บีทาเก้น','บีทาเกน'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โฟรโมสต์','โชคชัย','แดรี่โฮม','dairy home','เดรี่โฮม','อืมม มิลค์'
               ,'Umm! Milk','เอ็มมิลค์','mmilk','เอ็มมิ้ลค์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น'
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','สตอเบอรี่','สตรอเบอรี่','รสกาแฟ'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม'
               ,'high protein','Hi Protein','HIgh Protein','Whey Formula','วานิลลา'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์'
               ,'เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','เมจิ บัลแกเรีย','เมจิบัลแกเรีย','นมเปรี้ยว','รสธรรมชาติ','รสกลมกล่อม'
               ,'โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม','น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่'
               ,'bedtime','bed time','เบดไทม์','ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน'
               ,'7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11'
               ,'ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','แมคโคร','แม็คโคร','โลตัส','บิ๊กซี','bigc','Golden place','Big C'
               ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม'               
               ,'ซื้อประจำ','ซื้อไม่ได้','ของแถม','ราคาสูง','น้ำนมโค'
               ,'แพ้นม','แพ้นมวัว','เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก'
               ,'คีเฟอร์นม','พันทิป','ร้านนม','ซื้อไม่ได้','ลูกสุนัข','ลูกหมา','คายทิ้ง','เจมส์ จิ','เจมส์จิ','สตอรี่'
              ]
words = set(thai_words()).union(set(custom_list))
#custom_tokenizer = Tokenizer(engine='ulmfit', custom_dict=words, keep_whitespace=False)
trie = dict_trie(dict_source=words)

In [34]:
#https://github.com/PyThaiNLP/pythainlp-corpus/releases/tag/wiki_lm_lstm-v0.32

#from pythainlp.ulmfit import (replace_rep_after,fix_html,ungroup_emoji,replace_wrep_post,remove_space)

In [35]:
def tokenize_custom(x, cusDict):
    wtkn = word_tokenize(x, engine="newmm", custom_dict=cusDict, keep_whitespace=False)
    wtkn = checkStopwords(wtkn)
    wtkn = [x for x in wtkn if len(x) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [36]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "text": df.text[c],
            "token_text":df.token_text[c],
            "mention_product": df.mention_product[c],
            "mention_brand": df.mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [37]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "Table": df.Table[c],
            "URLs": lx,
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c]
        }
        listofdict.append(info)
    return listofdict

In [38]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','meiji','meji','miji']:
            ret_wtkn[i] = 'เมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟรโมสต์','โฟโมต','foremost']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['แดรี่โฮม','dairy home','เดรี่โฮม']:
            ret_wtkn[i] = 'แดรี่โฮม'
        elif each  in ['โชคชัย','อืมม มิลค์','umm milk','นมฟาร์มโชคชัย']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','ช็อค']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0%','0% fat','ขาดมันเนย']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','hi protein','ไฮโปรตีน','เวย์โปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิโกล์ดแม็กซ์','gold','gold max']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'lotus'
        elif each  in ['บิ๊กซี','bigc','Big C']:
            ret_wtkn[i] = 'bigc'
        elif each  in ['ท้องเสีย','ขี้แตก']:
            ret_wtkn[i] = 'ท้องเสีย'
        elif each  in ['ลูกสุนัข','ลูกหมา']:
            ret_wtkn[i] = 'ลูกสุนัข'
        elif each in ['ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต']:
            ret_wtkn[i] = 'ดาร์คช็อกโกแลต'
        elif each in ['bedtime','bed time','เบดไทม์']:
            ret_wtkn[i] = 'เบดไทม์'
        elif each in ['เมจิ บัลแกเรีย','เมจิบัลแกเรีย']:
            ret_wtkn[i] = 'เมจิบัลแกเรีย'
        elif each in ['โยเกิร์ต','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม']:
            ret_wtkn[i] = 'โยเกิร์ต'
        elif each in ['7-Eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7/11','7-11']:
            ret_wtkn[i] = '7-Eleven'
        elif each in ['บีทาเก้น','บีทาเกน']:
            ret_wtkn[i] = 'บีทาเก้น'
    return ret_wtkn

In [39]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','เมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค','หนองโพ','คาเนชั่น','บีทาเก้น']
    products = ['นม','สตรอว์เบอร์รี','ช็อกโกแลต','กาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','พาสเจอร์ไรส์','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','โยเกิร์ต','รสธรรมชาติ','รสกลมกล่อม','โยเกิร์ต','ซากุระ']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    else:
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [40]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

1.Tokenize comment

In [41]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x,trie))

In [42]:
df_process_comment.head()

,commentId,Table,URLs,text,Retrived_date,token_text
0,1,Meiji,https://pantip.com/topic/39868603,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู..."
1,2,Meiji,https://pantip.com/topic/39868603,เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน]
2,3,Meiji,https://pantip.com/topic/39868603,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-10-26 12:40:26.135083,"[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม..."
3,4,Meiji,https://pantip.com/topic/39868603,เป็นรสที่ขายไม่ดีครับ\r\nรสพื้นฐานคือ\r\nจืด แ...,2020-10-26 12:40:26.135083,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, %,, low..."
4,5,Meiji,https://pantip.com/topic/39868603,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-10-26 12:40:26.135083,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่..."


2.Generalized keyword (Brand & product)

In [43]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

3.Insert comments to DB

In [44]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

4.Tokenize head of thread

In [45]:
df_process_thread['token_text'] = df_process_thread['text'].apply(lambda x: tokenize_custom(x,trie))

5.Generalized keyword (Brand & product)

In [46]:
df_process_thread['token_text'] = df_process_thread['token_text'].apply(lambda x: generalize_keyword(x))

6.Mark domain/entity mention

In [47]:
df_process_thread['mention_brand'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['mention_product'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'p'))

In [48]:
df_process_thread

,commentId,URLs,text,token_text,mention_product,mention_brand
0,1,https://pantip.com/topic/39868603,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, 7-Eleven, ยังมี, นม, เมจิ, รสหวาน, ขาย, อ...","[นม, รสหวาน, นม, รสหวาน]",[เมจิ]
1,10,https://pantip.com/topic/39597709,ตามหัวข้อเลยค่ะ อยากรู้ว่านมเมจิที่ต้มก่อนนำไป...,"[หัวข้อ, อยากรู้, นม, เมจิ, ต้ม, ส่วนผสม, เครื...",[นม],[เมจิ]
2,18,https://pantip.com/topic/39747201,วันก่อนซื้อ นมเมจิขวด 2 ลิตร มากิน สังเกตฝามัน...,"[วันก่อน, ซื้อ, นม, เมจิ, ขวด, ลิตร, กิน, สังเ...","[นม, นม]",[เมจิ]
3,21,https://pantip.com/topic/39732615,อยากรู้ว่าMakroมีนมยี่ห้อ meiji(เมจิ) ขายมั้ยครับ,"[อยากรู้, makro, นม, ยี่ห้อ, เมจิ, เมจิ, ขาย]",[นม],"[เมจิ, เมจิ]"
4,28,https://pantip.com/topic/39005469,อยากถามว่า\r\n\r\n1.คิดค่าจัดส่งยังไงครับ\r\n2...,"[ถาม, 1., ค่าจัดส่ง, 2., ซื้อ, ได้, ส่วนลด, 3....",[],[]
5,33,https://pantip.com/topic/39928742,สวัสดี เราคือมนุษย์ออฟฟิศ อายุ 25 ปี ที่เสพติด...,"[สวัสดี, มนุษย์, ออฟฟิศ, อายุ, ปี, เสพติด, กิน...","[นม, ไขมัน 0%]",[เมจิ]
6,46,https://pantip.com/topic/39578663,ขอถามคุณผู้หญิงที่ซื้อนมไฮโปรตีนของยี่ห้อเมจิห...,"[ถาม, คุณผู้หญิง, ซื้อ, นม, ไฮโปรตีน, ยี่ห้อ, ...","[นม, ไฮโปรตีน, ไฮโปรตีน, ไฮโปรตีน, นม, ไฮโปรตีน]","[เมจิ, ดัชมิลล์, เมจิ, ดัชมิลล์]"
7,60,https://pantip.com/topic/38836146,😢,[],[],[]
8,66,https://pantip.com/topic/40190870,ผมอยู่ม.1ครับ ตอนป.6 ผมสูงเเค่ 142 เซน พอขึ้นม...,"[ผม, ม., ตอน, ป., ผม, แค่, เซน, ม., แต่, คนอื่...",[],[เมจิ]
9,73,https://pantip.com/topic/39151811,อยากได้รสที่เหมือนนมผงอะคะ ยังหาไม่เจอว่า จะใช...,"[อยากได้, รส, เหมือน, นมผง, อะ, หาไม่, เจอ, คร...","[นม, นม, นม]","[เมจิ, แมคโนเลีย]"


7.Insert thread to DB

In [49]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [50]:
del df_textList1,df_textList2,df_thread,df_Corpus,df_process_comment,df_process_thread

<br><br>
## 3.Process หัวกระทู้เพื่อดึง Domain (Brand & Product) ไม่ใช้โมเดลเทรนแล้วทำนาย ต้องการความถูกต้อง 100%

หากเคยบันทึกลง MongoDB แล้ว ให้เริ่มตรงนี้

In [51]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [52]:
df_urls_milk = df_thr_process[(df_thr_process.mention_brand.str.len() != 0) & (df_thr_process.mention_product.str.len() != 0)]
df_urls_milk['mention_brand'] = df_urls_milk['mention_brand'].apply(lambda x: repr(set(x)))
df_urls_milk['mention_product'] = df_urls_milk['mention_product'].apply(lambda x: repr(set(x)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
df_urls_milk.shape

(423, 6)

In [54]:
df_view = df_urls_milk[['URLs','mention_brand']]
df_view.column = ['URLs','mention_brand']
df_view_count = df_view.groupby('mention_brand').count()
print('จำนวนกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ในหัวกระทู้ =',df_view.shape[0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


จำนวนกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ในหัวกระทู้ = 423


In [55]:
df_view_count.sort_values(by='URLs',ascending=False)

,URLs
mention_brand,
{'เมจิ'},184
{'โฟร์โมสต์'},70
{'ดัชมิลล์'},50
{'โชคชัย'},24
"{'เมจิ', 'ดัชมิลล์'}",12
{'ไทยเดนมาร์ค'},8
"{'คาเนชั่น', 'เมจิ'}",6
"{'โฟร์โมสต์', 'เมจิ'}",6
"{'โฟร์โมสต์', 'ดัชมิลล์'}",5


In [56]:
#aa = df_urls_milk[(df_urls_milk['mention_brand']=="{'เมจิ'}")&(df_urls_milk['URLs']=='https://pantip.com/topic/31061329')].token_text
#aa.iloc[0]

In [57]:
#df_urls_milk.to_excel("Docs/วิเคราะห์ข้อมูล/domain.xlsx",index=False)

### 4.ทำ Topic Modeling และคัดเลือกกระทู้ที่มีความเกี่ยวข้อง แล้วดึงคำไปใช้

In [115]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [116]:
df_join_url = pd.merge(df_cmt_process,df_urls_milk,how='inner',on='URLs')
df_join_url = df_join_url[['Retrived_date','URLs','commentId','mention_brand', 'mention_product','token_text_x']]
df_join_url.columns = ['Retrived_date','URLs','commentId','mention_brand', 'mention_product','token_text']

In [88]:
df_cmt_process.shape

(60545, 7)

In [89]:
df_join_url.tail()

,Retrived_date,URLs,commentId,mention_brand,mention_product,token_text
7947,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65703,{'โฟร์โมสต์'},"{'รสจืด', 'นม'}","[ปกติ, ขับ, ออกจาก, แยก, ไม่, ได้, เลี้ยว, ไม่..."
7948,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65704,{'โฟร์โมสต์'},"{'รสจืด', 'นม'}","[เสียใจ, รถ, ถนน, , ไม่, , ระวัง, ตัวเอง, ดีกว..."
7949,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65705,{'โฟร์โมสต์'},"{'รสจืด', 'นม'}","[หน้า, ห้าง, ..., วาง, กรวย, หา, แพะ]"
7950,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65706,{'โฟร์โมสต์'},"{'รสจืด', 'นม'}","[ขี่, รถ, ผ่านหน้า, big c, รัชดา, ผม, วิ่ง, เล..."
7951,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65707,{'โฟร์โมสต์'},"{'รสจืด', 'นม'}","[ชีวิต, เลือก, ได้, ไม่, ขับ, มอไซด์, เสี่ยง, ..."


ถ้า EDA แล้วให้ Skip ส่วนนี้ ข้ามไปส่วนหลัง Reduce Term (5) ได้เลย

In [90]:
df_view = df_join_url[['commentId','mention_brand']]
df_view.column = ['commentId','mention_brand']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [91]:
df_view_count = df_view.groupby('mention_brand').count()

In [92]:
print('จำนวน comment ในกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ =',df_view.shape[0])

จำนวน comment ในกระทู้ที่มีการพูดถึงนมหรือชื่อแบรนด์ = 7952


In [93]:
df_view_count.sort_values(by='commentId',ascending=False)

,commentId
mention_brand,
{'เมจิ'},3481
{'ดัชมิลล์'},1195
{'โฟร์โมสต์'},1078
"{'เมจิ', 'ดัชมิลล์'}",506
{'โชคชัย'},234
{'ไทยเดนมาร์ค'},137
"{'โฟร์โมสต์', 'ดัชมิลล์'}",132
"{'โฟร์โมสต์', 'ไทยเดนมาร์ค', 'ดัชมิลล์', 'หนองโพ'}",117
"{'โฟร์โมสต์', 'เมจิ'}",116


In [117]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text'])

In [95]:
#dictionary.token2id.keys()

In [118]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in df_join_url['token_text']]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [97]:
#num_topics = 9
chunksize = 4000                  # size of the doc looked at every pass
iterations = 50
eval_every = 1                    # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize \
                                     ,alpha='auto', eta='auto',iterations=iterations, eval_every=eval_every)
                                     #,num_topics=num_topics \
                                     #, eval_every=eval_every)

Wall time: 23.2 s


In [70]:
#pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, R=40, lambda_step=0.1)

### 5.get_term_topics เทียบคำที่ได้ไป Review Literature ตามค่าความน่าจะเป็น แล้วสรุป Term

https://radimrehurek.com/gensim/models/ldamodel.html

<b>Domain ที่พบหลักๆ นอกเหนือจากตัว Product ห้างที่ขาย แบรนด์ (อย่างไรก็ตามต้องทำ LDA Vis อีกรอบเพื่อจัดกลุ่มใหม่หลังคัดเลือก Topic/Term)</b><br>
ค้นหาเทียบเคียงกับ ปัจจัยที่มีผลต่อการบริโภคนม
1. แม่และเด็ก (Kurajdova et al, 2015)
2. ออกกำลังกาย เล่นเวท วิ่ง พูดถึง Whey หางนม โปรตีน เป็นหลัก (คิดว่าแตกจากหมวดสุขภาพ)
3. โปรโมชั่น เช่น จัดโปร ส่วนลด พันแถม แพ๊กคู่ (สุภชาติ ชัยณรงค์สิงห์, 2539)
4. เอานมให้หมาแมวกิน (เทรนด์ใหม่)
5. รสชาติและคุณภาพ เช่น หอม เหม็น มัน อร่อย บูด เสีย GMP (ชาลิสา สถีระกานนท์, 2560)
6. ความยากง่ายในการหาซื้อ (Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
7. ทำกาแฟ ทำขนม (Kurajdova et al, 2015)
8. สุขภาพ ความสูง ส่วนผสมเพื่อสุขภาพ การขับถ่าย อนุมูลอิสระ (Kurajdova et al, 2015 & Aen De Alwis et al, 2009 & สุภชาติ ชัยณรงค์สิงห์, 2539)
9. ราคาและความคุ้มค่า (ปิยฉัตร ช่างเหล็ก, 2561)
<br><br>ตรงนี้ต้องหา Paper มาช่วยยันในบางหัวข้อ ถ้ามีส่วนเกินถือว่าเป็นบริบทของประเทศไทย

In [72]:
#Get the most relevant topics to the given word.
# Set probability = 0.0001

def get_terms_topics(term_text, d, model, minProb=0.0001):
    listofTup = []
    for tt_couple in model.get_term_topics(dictionary.token2id[term_text], minimum_probability=minProb):
        id_, prob = tt_couple
        listofTup.append((d[id_], prob))
    df = pd.DataFrame(listofTup)
    df.insert(0,'Term',value=term_text)
    df.columns = ['Term','topic','prob']
    return df.sort_values(by='prob', ascending=False)

In [73]:
#Get the most relevant topics to the given word.
# Set probability = 0.0001

def get_topics_terms(topic, d, model, minProb=0.0001):
    listofTup = []
    for tt_couple in model.get_topic_terms(topic,topn=30):
        id_, prob = tt_couple
        listofTup.append((d[id_], prob))
    df = pd.DataFrame(listofTup)
    df.insert(0,'Topic',value=d[topic])
    df.columns = ['Topic','term','prob']
    return df.sort_values(by='prob', ascending=False)

In [74]:
# องค์ประกอบพื้นฐาน ตาม master ที่ใช้ตัดคำ และ Generalized
#get_terms_topics('ซื้อ',dictionary,model)
#get_terms_topics('ขาย',dictionary,model)

# ไล่เช็คทีละ Topic โดยเลือกจาก Top30 Prob แต่ละกลุ่ม และดูตัวที่สนใจ (ตามทบทวนวรรณกรรม)
# สุดที่ 99 กลุ่ม
get_topics_terms(99,dictionary,model).head(3)

,Topic,term,prob
0,ชั้น,ผม,0.032077
1,ชั้น,ไม่,0.025749
2,ชั้น,,0.020873


In [98]:
attrributes = ['การบูด','โปร','โปรโมชั่น','แถม','บูด','เป็นผลดี','หา','หมดอายุ', 'กลิ่น','หอม','กลิ่นหอม', 'แนะนำ', 'กิน', 'shelf life','แม่','คุณแม่'
 ,'ลูก','เด็ก','เลี้ยงลูก','เลี้ยงเด็ก','พัฒนาการ','คุณแม่มือใหม่','สั่ง', 'ค่าใช้จ่าย', 'เพิ่มขึ้น', 'สินค้า', 'ชอบ','นิยม', 'สี', 'ซื้อไม่ได้'
 , 'ทิ้ง', 'จืด','หวาน' , 'fat', 'นิยม','ราคาสูง','กาแฟ','ชงกาแฟ','หาย','ต้ม','อุ่น','ร้อน', 'พื้นฐาน', 'ขายไม่ดี'
 , 'กาแฟสด', 'จ่าย', 'ไขมันต่ำ', 'นม', 'ขาย', 'รส','รสชาติ', 'รสหวาน','กลมกล่อม','ไขมัน','พนักงาน','ไมโครเวฟ','ฟาร์ม'
 ,'ส่วนผสม','ผสม','เบาหวาน','รสพื้นฐาน','โปรตีน','เวย์','เวท','วิ่ง','protein','whey','ขับถ่าย','อึ','dha','น้ำตาล','เบาหวาน'
 ,'ฝา', 'นมสด','ถ้วย', 'ร้าน', 'ดื่ม', 'เซเว่น', 'ขวด', 'นึกถึง', 'เสียใจ', 'แตก', 'ซื้อ','ขาย','มีลูก','เทรนเนอร์'
 ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม','แพ้นม','แพ้นมวัว','เล่นเวท','ฝาน้ำเงิน','ฝาสีเขียว'
 ,'นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ลูกสุนัข','สุนัข','หมา','แมว','คายทิ้ง','เจมส์จิ','สตอรี่','แป้ง','ลดราคา'
 ,'ซื้อประจำ','ซื้อไม่ได้','คาปูชิโน่','อเมริกาโน่','ร้านนม','whey formula','ผิดสังเกต','เสียความรู้สึก','ชี้แจง','บำรุง','น้ำผึ้ง'
 ,'อาหารเสริม','มีประโยชน์','วิตามิน','นมผง','แคลเซี่ยม','ท้อง','แพะ','อร่อย','โภชนาการ','โรงเรียน','พ่อแม่','ครู','ยูเอชที'
 ,'ผู้บริโภค','ขโมย','ไอโอดีน','นมข้นหวาน','เนย','ตรวจสอบ','แกลลอน','มันดี','นมวัว','อาเจียน','เวฟ','ไมโครเวฟ','สารอาหาร'
 ,'ขนส่ง','ถูก','แพง','เค้ก','ทิ้ง','วันหมดอายุ','โอเมก้า','กล่อง','พลังงาน','โภชนาการ','ขนมปัง','ของแถม','ราคาสูง','น้ำนมโค'
 ,'บรรจุภัณฑ์','นมถั่วเหลือง','ความแข็งแรง','แข็งแรง','พรีเซ็นเตอร์','ญี่ปุ่น','น้ำผลไม้','ออกกำลังกาย','ประหยัด','วิปปิ้ง','uht','เชื้อจุลินทรีย์'
 ,'แลคโตส']

products = ['สตรอว์เบอร์รี','ช็อกโกแลต','กาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
,'รสกล้วย','grass fed','นมฟรีแลคโตส','พาสเจอร์ไรส์','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','โยเกิร์ต','รสธรรมชาติ','รสกลมกล่อม','โยเกิร์ต','ซากุระ','โคล่า'
,'แตงโม']

brands = ['ดัชมิลล์','เมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค'
          ,'หนองโพ','คาเนชั่น','บีทาเก้น','จิตรลดา']
stores = ['tops','makro','lotus','bigc','7-Eleven']

reduceCol = attrributes + products + brands + stores

In [110]:
len(reduceCol)

221

In [149]:
def reduced_keyword(wtkn,redCol):
    del_list = []
    ret_wtkn = wtkn
    for each in ret_wtkn:
        if each not in redCol:
            del_list.append(each)
    ret_wtkn = [x for x in ret_wtkn if x not in del_list]
    return ret_wtkn

In [152]:
df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x,reduceCol))

In [153]:
df_join_url.head()

,Retrived_date,URLs,commentId,mention_brand,mention_product,token_text,token_text_reduce
0,2020-10-26 12:40:26.135083,https://pantip.com/topic/39868603,1,{'เมจิ'},"{'รสหวาน', 'นม'}","[7-Eleven, นม, เมจิ, รสหวาน, ขาย, หาย, หา, top...","[7-Eleven, นม, เมจิ, รสหวาน, ขาย, หาย, หา, top..."
1,2020-10-26 12:40:26.135083,https://pantip.com/topic/39868603,2,{'เมจิ'},"{'รสหวาน', 'นม'}",[เบาหวาน],[เบาหวาน]
2,2020-10-26 12:40:26.135083,https://pantip.com/topic/39868603,3,{'เมจิ'},"{'รสหวาน', 'นม'}","[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม...","[หา, ซื้อไม่ได้, นิยม, ขาย, ซื้อ, กิน, รสหวาน]"
3,2020-10-26 12:40:26.135083,https://pantip.com/topic/39868603,4,{'เมจิ'},"{'รสหวาน', 'นม'}","[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, %,, ไขม...","[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, ไขมันต่ำ, ไขม..."
4,2020-10-26 12:40:26.135083,https://pantip.com/topic/39868603,5,{'เมจิ'},"{'รสหวาน', 'นม'}","[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่...","[ขายไม่ดี, shelf life, หมดอายุ, ค่าใช้จ่าย, ร้..."


ตัดคำแล้วทำ LDA อีกรอบเพื่อดู feature ที่ชัดเจนขึ้น ก่อนสร้าง Bag of word เพื่อออกกราฟ & MDS

## สร้าง Bag of word dataframe

### Create co-occurrence matrix จาก Bag of word

### Create co-occurrence matrix with Lift normalization